In [1]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy #uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering libraryt requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe



print('Libraries imported.')

Libraries imported.


In [125]:
# The code was removed by Watson Studio for sharing.

,Unnamed: 0,Neighbourhood,Avg Price per sqft
0,0,AP SE Board Hyderabad,4445.0
1,1,Ac Guards Hyderabad,6492.0
2,2,Ahmed Nagar Hyderabad,2343.0
3,3,Amberpet Hyderabad,2222.0
4,4,Ameerpet Hyderabad,9628.0


In [128]:
hyderabad_neighbourhoods=df_data_3.dropna(axis=0,how='any')
hyderabad_neighbourhoods.head()


,Unnamed: 0,Neighbourhood,Avg Price per sqft
0,0,AP SE Board Hyderabad,4445.0
1,1,Ac Guards Hyderabad,6492.0
2,2,Ahmed Nagar Hyderabad,2343.0
3,3,Amberpet Hyderabad,2222.0
4,4,Ameerpet Hyderabad,9628.0


In [155]:
hyderabad_neighbourhoods=hyderabad_neighbourhoods[hyderabad_neighbourhoods.Neighbourhood != 'Chaitanyapuri Colony Hyderabad']
#hyderabad_neighbourhoods.drop(['Unnamed: 0'],axis=1,inplace = True)


### Locating Google Office on Map ###

In [169]:
office_address = 'Kondapur Village, Hyderabad'

geolocator = Nominatim(user_agent="hyd_explorer")
location = geolocator.geocode(office_address)
office_latitude = location.latitude
office_longitude = location.longitude
print('The geograpical coordinate of Google office are {}, {}.'.format(office_latitude, office_longitude))


The geograpical coordinate of Google office are 17.4587912, 78.3730556.


In [180]:
# create map of Hyderabad showing Google office using latitude and longitude values
map_Hyderabad = folium.Map(location=[office_latitude, office_longitude], zoom_start=12)

# add markers to map
label = 'Google'
folium.Marker(
    [office_latitude, office_longitude],
    radius=10,
    popup=label,
    icon=folium.Icon(color='red')
    ).add_to(map_Hyderabad)  
    
map_Hyderabad

### Now we shall get the latitude and longitudes of all the neighbourhoods and locate them on the map ###



In [168]:
latitude=[]
longitude =[]

for i in range(len(hyderabad_neighbourhoods)):
    geolocator = Nominatim(user_agent="latlan_explorer")
    location = geolocator.geocode(hyderabad_neighbourhoods.iloc[i,0])
    latitude.append(location.latitude)
    longitude.append(location.longitude)

hyderabad_neighbourhoods['latitude'] = latitude
hyderabad_neighbourhoods['longitude'] = longitude
hyderabad_neighbourhoods

,Neighbourhood,Avg Price per sqft,latitude,longitude
0,AP SE Board Hyderabad,4445.0,17.493930,78.402257
1,Ac Guards Hyderabad,6492.0,17.399877,78.458051
2,Ahmed Nagar Hyderabad,2343.0,17.404066,78.444229
3,Amberpet Hyderabad,2222.0,17.390263,78.516481
4,Ameerpet Hyderabad,9628.0,17.437501,78.448251
5,Anand Nagar Hyderabad,2545.0,17.313960,78.616579
6,Ashok Nagar Hyderabad,6106.0,17.405826,78.490260
7,Asif Nagar Hyderabad,5049.0,17.383623,78.446144
8,BK Guda Hyderabad,3034.0,17.446628,78.446289
9,Banjara Hills Hyderabad,29600.0,17.417746,78.439901


In [182]:
# create map of Hyderabad showing Google office using latitude and longitude values
map_Hyderabad = folium.Map(location=[office_latitude, office_longitude], zoom_start=10)

# add markers to map
folium.Marker(
    [office_latitude, office_longitude],
    radius=10,
    icon=folium.Icon(color='red'),
    popup='Google'
).add_to(map_Hyderabad)

for lat, lng, neighborhood in zip(hyderabad_neighbourhoods['latitude'] ,hyderabad_neighbourhoods['longitude'] ,hyderabad_neighbourhoods['Neighbourhood']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=neighborhood,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map_Hyderabad)
map_Hyderabad

### Calculate the distance from each neighbourhood to office ###

In [181]:
# Importing the geodesic module from the library 
from geopy.distance import geodesic 

distance_from_office = []

for i in range(len(hyderabad_neighbourhoods)):
    distance_from_office.append(geodesic((office_latitude,office_longitude),(hyderabad_neighbourhoods.iloc[i,2],hyderabad_neighbourhoods.iloc[i,3])).km)

hyderabad_neighbourhoods['Distance in kms'] = distance_from_office
hyderabad_neighbourhoods

,Neighbourhood,Avg Price per sqft,latitude,longitude,Distance in kms
0,AP SE Board Hyderabad,4445.0,17.493930,78.402257,4.974308
1,Ac Guards Hyderabad,6492.0,17.399877,78.458051,11.137903
2,Ahmed Nagar Hyderabad,2343.0,17.404066,78.444229,9.687996
3,Amberpet Hyderabad,2222.0,17.390263,78.516481,17.021086
4,Ameerpet Hyderabad,9628.0,17.437501,78.448251,8.328190
5,Anand Nagar Hyderabad,2545.0,17.313960,78.616579,30.440074
6,Ashok Nagar Hyderabad,6106.0,17.405826,78.490260,13.762453
7,Asif Nagar Hyderabad,5049.0,17.383623,78.446144,11.380137
8,BK Guda Hyderabad,3034.0,17.446628,78.446289,7.894918
9,Banjara Hills Hyderabad,29600.0,17.417746,78.439901,8.429991
